In [2]:
import os
import dotenv

import streamlit as st
from openai import OpenAI
import requests

In [3]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")  # if the key already exists in the environment variables, it will use that, otherwise it will use the .env file to get the key
if not OPENAI_API_KEY:
    dotenv.load_dotenv(".env")
    OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [ ]:
def identify_domain(ps_pair):
    
    client = OpenAI(
        api_key = OPENAI_API_KEY
    )
    
    prompt_text = """
    You are a text classifier model. Given the following domains, identify which domains of expertise the following problem solution pairs fall into and output it by just separating each with commas.
    
    Domains:
    
    """

In [35]:
# Create prompt to evaluate an idea
def prompt_for_expert_idea_evaluation(category, ps_pair, rubric):
    
    client = OpenAI(
        api_key = OPENAI_API_KEY
    )

    # Format the prompt to ask GPT to pretend to be an expert in the given category
    prompt_text = f"""
    
    You are an expert in {category}. You have been asked to evaluate the following business idea containing an identified problem and solution:
    
    {ps_pair}
    
    Based on the following rubric in a CSV format, rate the idea on each of the criteria (where row 1 is the option of points per criteria and the remaining rows represents the criteria itself) based on the point system defined: 
    
    {rubric}
    
    The output should a rating for each criteria with a colon indicating the reason behind that chosen rating. Separate each criteria with a newline.
    
    """

    # Call the OpenAI API
    response = client.chat.completions.create(
      model="gpt-3.5-turbo",  # Choose the appropriate engine
      messages=[{"role": "user", "content": prompt_text}],
    )

    # Return the text part of the response
    return response.choices[0].message.content

In [33]:
sample_rubric = """
Originality, Idea has common elements with no unique differentiation. (1), Idea shows some novel thinking and differentiation. (2), Idea is largely original, showing significant new thinking. (3), Idea is completely unique demonstrating groundbreaking thinking. (4)
Feasibility, Idea has significant practical or technical obstacles. (1), Idea is somewhat practical but faces notable challenges. (2), Idea is fairly practical with manageable challenges. (3), Idea is highly practical and can be implemented smoothly. (4)
Impact, Idea has a limited or unclear impact. (1), Idea has a moderate impact with some tangible benefits. (2), Idea has a significant impact with clear benefits. (3), Idea has a transformative impact with far-reaching benefits. (4)
Development & Research, Idea is underdeveloped with minimal research or supporting data. (1), Idea is somewhat developed with some research or supporting data. (2), Idea is well-developed with substantial research or supporting data. (3), Idea is fully developed with extensive research or supporting data. (4)
Scalability, Idea shows little to no potential for growth or adaptation. (1), Idea shows some potential for growth or adaptation. (2), Idea shows considerable potential for growth or adaptation. (3), Idea shows extensive potential for growth or adaptation. (4)
"""

sample_ps_pair = """
Problem: Single-use plastic packaging has become an all-too-common sight in our environment, resulting in detrimental environmental impacts. Studies show that approximately 8.3 billion tonnes of plastic have been produced since 1950, and 60% of that plastic ends up in either our landfills or the natural environment. This contributes greatly to pollution, harms wildlife, affects human health, and exacerbates climate change.   
Solution: To alleviate this problem, we propose developing a consumer-friendly, easy-to-use app that manages and optimises the reverse logistics of packaging. Imagine this: when purchasing a product, consumers will pay a one-time fee for reusable packaging. After use, they can return the empty packaging via designated collection points or direct pickup services. The app will facilitate this process by tracking, scheduling pickups or locating nearby collection points. The packaging is then cleaned and reused, thereby reducing the need for new plastic production and ensuring the packaging circulates within the economy, instead of ending up as waste. This app will not only help consumers make more sustainable choices but also incentivise producers to shift towards a circular economy model.
"""

In [37]:
print(prompt_for_expert_idea_evaluation("Waste Management", sample_rubric, sample_ps_pair))

Feasibility: 3 - The idea is fairly practical with manageable challenges. The development of an app for managing and optimizing the reverse logistics of packaging is technically feasible and can be implemented smoothly. However, there may be challenges in terms of coordinating pickup services and establishing collection points.

Originality: 2 - The idea shows some novel thinking and differentiation. While the concept of reusable packaging and reverse logistics is not entirely unique, the use of a consumer-friendly app to facilitate the process is a relatively novel approach.

Impact: 3 - The idea has a significant impact with clear benefits. The app has the potential to reduce the amount of single-use plastic packaging that ends up in landfills or the natural environment. This would lead to a reduction in pollution, protection of wildlife, improved human health, and positive effects on climate change.

Development & Research: 2 - The idea is somewhat developed with some research or su

In [ ]:
def extract_scores():

In [ ]:
def prompt_for_combining_opinions():
    

In [ ]:
def prompt_for_business_team_evaluation(roles, ps_pair, rubric):